In [1]:
import numpy as np
import sys
import torch
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

In [2]:
def create_shape(dim, j):
    assert(j in [0, 1, 2, 3])
    assert(dim in [2, 3])
    assert(j <= dim)
    if dim == 3:
        V = [[1, 1, 1],
             [1, -1, -1],
             [-1, 1, -1],
             [-1, -1, 1]]
        if j == 0:
            E = [[0],
                 [1],
                 [2],
                 [3]]
        elif j == 1:
            E = [[0,1],
                 [0,2],
                 [0,3],
                 [1,2],
                 [1,3],
                 [2,3]]
        elif j == 2:
            E = [[0,1,2],
                 [0,1,3],
                 [0,2,3],
                 [1,2,3]]
        elif j == 3:
            E = [[0,1,2,3]]
    elif dim == 2:
        V = [[.25, .25],
             [.75, .25],
             [.50, .75]]
        if j == 0:
            E = [[0],
                 [1],
                 [2]]
        elif j == 1:
            E = [[0,1],
                 [0,2],
                 [1,2]]
        elif j == 2:
            E = [[0,1,2]]
    V = torch.FloatTensor(V)
    E = torch.LongTensor(E)
    
    # normalize V
    V_bb = torch.max(V, dim=0)[0] - torch.min(V, dim=0)[0]
    V_c = (torch.max(V, dim=0)[0] + torch.min(V, dim=0)[0]) / 2
    V -= V_c
    V /= 1.5*V_bb
    V += 0.5
    print(V_bb)
    return V, E

In [3]:
# random hull
def rand_hull(n_points, dim):
    V = np.random.rand(n_points, dim)
    mesh = Delaunay(V)
    E = mesh.simplices
    return torch.FloatTensor(V), torch.LongTensor(E)

In [4]:
def simplex_content(V, E):
    """
    Compute the content of simplices in a simplicial complex
    :param V: vertex tensor. float tensor of shape (n_vertex, n_dims)
    :param E: element tensor. int tensor of shape (n_elem, j+1)
    """
    ne = E.shape[0] # number of elements
    nppe = E.shape[1] # number of points per element
    assert(nppe in [1, 2, 3, 4]) # points, lines, tri or tet
    if nppe == 1: # points
        return torch.ones(ne, 1, dtype=V.dtype)
    if nppe == 2: # lines
        P = V[E]
        Len = torch.norm(P[:, 1:] - P[:, :-1], dim=-1)
        return Len
    elif nppe == 3: # triangles
        E_ = torch.cat([E, E[:, 0:1]], dim=-1)
        P = V[E_]
        L = torch.norm(P[:, 1:] - P[:, :-1], dim=-1)
        S = torch.sum(L, dim=-1, keepdim=True) / 2
        Area = torch.sqrt(S*(S-L[:, 0:1])*(S-L[:, 1:2])*(S-L[:, 2:])) # Heron's Formula
        return Area
    elif nppe == 4: # tetrahedron
        P = V[E]
        Va = P[:, 1] - P[:, 0]
        Vb = P[:, 2] - P[:, 0]
        Vc = P[:, 3] - P[:, 0]
        Vol = torch.abs(torch.einsum('ab,ab->a', (Va, torch.cross(Vb, Vc, dim=-1))) / 6)
        return Vol
        

In [5]:
import torch
from torch.autograd import Function

from utils import fftfreqs, simplex_content, triangulate_interior, permute_seq
from math import ceil, factorial


class DDSL(Function):
    """
    Fourier transform for signal defined on a j-simplex set in R^n space
    :param V: vertex tensor. float tensor of shape (n_vertex, n_dims)
    :param E: element tensor. int tensor of shape (n_elem, j or j+1)
              if j cols, triangulate/tetrahedronize interior first.
    :param D: int ndarray of shape (n_vertex, n_channel)
    :param res: n_dims int tuple of number of frequency modes
    :param t: n_dims tuple of period in each dimension
    :param j: dimension of simplex set
    :param mode: normalization mode.
                 'density' for preserving density, 'mass' for preserving mass
    :return: F: ndarray of shape (res[0], res[1], ..., res[-1]/2, n_channel)
                last dimension is halfed since the signal is assumed to be real
    """
    @staticmethod
    def forward(ctx, V, E, D, res, t, j, elem_batch=100, mode='density'):
        ## boiler-plate
        ctx.res = res
        ctx.t = t
        ctx.j = j
        ctx.mode = 1 if mode == 'density' else 0
        ctx.save_for_backward(V, E, D)

        ## compute frequencies F
        n_dims = V.shape[1]
        assert(n_dims == len(res))  # consistent spacial dimensionality
        assert(E.shape[0] == D.shape[0])  # consistent vertex numbers
        assert(mode in ['density', 'mass'])

        # number of columns in E
        subdim = E.shape[1] == j and n_dims == j
        assert (E.shape[1] == j+1 or subdim)
        if subdim:
            E = triangulate_interior(V, E)
        n_elem = E.shape[0]
        n_vert = V.shape[0]
        n_channel = D.shape[1]

        # frequency tensor
        omega = fftfreqs(res).to(V.device) # [dim0, dim1, dim2, d]
        omega[tuple([0] * n_dims)] += 1e-3 # will get rid of this

        # normalize frequencies
        for dim in range(n_dims):
            omega[..., dim] *= 2 * np.pi / t[dim]

        # initialize output F
        F_shape = list(omega.shape)[:-1]
        F_shape += [n_channel, 2]
        F = torch.zeros(*F_shape, dtype=torch.float32, device=V.device) # [dimX, dimY, dimZ, n_chan, 2] 2: real/imag

        # compute content array
        C = simplex_content(V, E) # [n_elem, 1]
        print(C.shape)

        # compute element-point tensor
        P = V[E] # [n_elem, j+1, d]

        # loop over element batches
        for idx in range(ceil(n_vert/elem_batch)):
            id_start = idx * elem_batch
            id_end = min((idx+1) * elem_batch, n_elem)
            X = P[id_start:id_end] # [elem_batch, j+1, d]
            Di = D[id_start:id_end] # [elem_batch, n_channel]
            Ci = C[id_start:id_end] # [elem_batch, 1]
            CDi = Ci * Di # [elem_batch, n_channel]
            sig = torch.einsum('bjd,...d->bj...', (X, omega)) 
            sig = torch.unsqueeze(sig, dim=-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1]
            sig_sub = [[None] * (n_dims+1)] * (n_dims+1)
            esig = torch.stack((torch.cos(sig), torch.sin(sig)), dim=-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1, 2]
            sig = torch.unsqueeze(sig, dim=-1) # [elem_batch, j+1, dimX, dimY, dimZ, 1, 1]
            denom = torch.ones_like(sig) # [elem_batch, j+1, dimX, dimY, dimZ, 1, 1]
            for dim in range(1, j+1):
                seq = permute_seq(dim, j+1)
                denom *= sig - sig[:, seq]
            tmp = torch.sum(esig / denom, dim=1) # [elem_batch, dimX, dimY, dimZ, 1, 2]
            CDi.unsqueeze_(-1)
            for _ in range(n_dims):
                CDi.unsqueeze_(dim=1) # [elem_batch, 1, 1, 1, n_channel, 1]
            tmp *= CDi # [elem_batch, dimX, dimY, dimZ, n_channel, 2]
            Fi = torch.sum(tmp, dim=0, keepdim=False) # [dimX, dimY, dimZ, n_channel, 2]
            Fi[tuple([0] * n_dims)] = - 1 / factorial(j) * torch.sum(CDi, dim=0).unsqueeze(dim=-1)
            # Fi *= 1j**j # [dimX, dimY, dimZ, n_chan, 2] 2: real/imag
            if j == 0:
                pass
            elif j == 1:
                Fi = Fi[..., [1, 0]]
                Fi[..., 0] = -Fi[..., 0]
            elif j == 2:
                Fi *= -1
            elif j == 3:
                Fi = Fi[..., [1, 0]]
                Fi[..., 1] = -Fi[..., 1]
            F += Fi

        if mode == 'density':
            if not np.array_equal(res, res[0]*np.ones(len(res))):
                print("WARNING: density preserving mode not correctly implemented if not all res are equal")
            F *= res[0] ** j
        return F

    @staticmethod
    def backward(ctx, dF):
        V, E, D = ctx.saved_tensors
        dV = ddsl_cuda.backward(dF, V, E, D, res, t, j, mode=ctx.mode)
        return dV, None, None, None, None, None, None


In [6]:
dim = 3
j = 3
npoints = 5
r = 64

# V, E = create_shape(dim=dim,j=j)
V, E = rand_hull(npoints, dim)
# V += 1e-3 * torch.rand(V.shape)
D = torch.ones(E.shape[0], 1, dtype=V.dtype)

res = [r] * dim
t = [1] * dim

# V, E, D = V.cuda(), E.cuda(), D.cuda()

nuft = DDSL.apply
from time import time
t0 = time()
F = nuft(V, E, D, res, t, j, 100, "density")
print(time() - t0)

torch.Size([3, 1])
0.212738037109375


In [ ]:
V, E = rand_hull(100, 3)
simplex_content(V, E).shape


In [ ]:
F = F.squeeze()
F_ = F.cpu().detach().numpy()
f_ = torch.irfft(F, dim, signal_sizes=res).squeeze().cpu().detach().numpy()
print(F_.shape)
print(f_.shape)
plt.figure()
plt.imshow(f_[80, :, :]*8)
plt.colorbar()
plt.show()

In [ ]:
print(V)